# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 가능


In [3]:
!nvidia-smi

Fri Sep 17 15:19:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla M60           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P8    14W / 150W |      3MiB /  7618MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 Samsung Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_samsung.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,삼성전자,5930,매일경제,2021010104,[반도체] 메모리값 강세 지속…파운드리 수요도 늘듯,http://news.mk.co.kr/newsRead.php?no=387&year=...,◆ 신년기획 2021 경제기상도 ◆\r\n\n\n이재용 삼성전자 부회장(왼쪽)이 지...,2021-01-04,4,81000,84400,80200,83000,38655276,0.024691,1,0,신년 기획 경제 기상도 이재용 삼성 전자 부회장 왼쪽 지난해 충남 온양 삼성 전자 ...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2021 = pos_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <가치 있는> & <언택트> : 0.5496575236320496
***2021-01-04 Cosine Similarity between <개선> & <향상> : 0.5878475308418274
***2021-01-04 Cosine Similarity between <도움이 되는> & <언택트> : 0.5378727316856384
***2021-01-04 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2021-01-04 Cosine Similarity between <의심의 여지가 없는> & <언택트> : 0.529306173324585
***2021-01-04 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-04 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2021-01-04 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-04 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <증대> & <향상> : 0.6337747573852539
***2021-01-04 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-04 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-04 Cosine Similarity 

***2021-01-05 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-01-05 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-05 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-05 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2021-01-05 Cosine Similarity between <최선> & <최선> : 0.9999999403953552
***2021-01-05 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-01-05 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-01-05 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-05 Cosine Similarity between <가치 있는> & <박유악> : 0.5374411940574646
***2021-01-05 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2021-01-05 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-01-05 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-05 Cosine Similarity between <도움이 되는> & <박유악> : 0.5079639554023743
***2021-01-05 Cosine Similarity between <본

***2021-01-06 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-06 Cosine Similarity between <확보> & <마련> : 0.5137236714363098
***2021-01-06 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2021-01-06 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-06 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2021-01-06 Cosine Similarity between <가치 있는> & <홈테인먼트> : 0.5342689752578735
***2021-01-06 Cosine Similarity between <개선> & <향상> : 0.5878475308418274
***2021-01-06 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-06 Cosine Similarity between <도움이 되는> & <홈테인먼트> : 0.6065888404846191
***2021-01-06 Cosine Similarity between <본격적인> & <본격> : 0.6104379296302795
***2021-01-06 Cosine Similarity between <실현> & <구현> : 0.5969869494438171
***2021-01-06 Cosine Similarity between <의심의 여지가 없는> & <홈테인먼트> : 0.5886704325675964
***2021-01-06 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-06 Cosine Simil

***2021-01-07 Cosine Similarity between <적극적으로> & <적극> : 0.6639532446861267
***2021-01-07 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-07 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2021-01-07 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-07 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2021-01-07 Cosine Similarity between <증대> & <향상> : 0.6337747573852539
***2021-01-07 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2021-01-07 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-07 Cosine Similarity between <추진> & <발전> : 0.5099795460700989
***2021-01-07 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-01-07 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-01-07 Cosine Similarity between <충족> & <요구> : 0.5395624041557312
***2021-01-07 Cosine Similarity between <충족> & <충족> : 0.9999999403953552
***2021-01-07 Cosine Similarity between <해결> & <

***2021-01-11 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2021-01-11 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-11 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2021-01-11 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-11 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-11 Cosine Similarity between <충족> & <요구> : 0.5395624041557312
***2021-01-11 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-01-11 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-11 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2021-01-11 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-11 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2021-01-11 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2021-01-11 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-11 Cosine Similarity between <추진> & <진행> 

...

***2021-09-08 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-08 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-08 Cosine Similarity between <추진> & <발전> : 0.5099795460700989
***2021-09-08 Cosine Similarity between <충족> & <요구> : 0.5395624041557312
***2021-09-08 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2021-09-08 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2021-09-08 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-09-08 Cosine Similarity between <장점> & <특징> : 0.6442348957061768
***2021-09-08 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-08 Cosine Similarity between <본격적인> & <본격> : 0.6104379296302795
***2021-09-08 Cosine Similarity between <장점> & <장점> : 0.9999999403953552
***2021-09-08 Cosine Similarity between <최선> & <최선> : 0.9999999403953552
***2021-09-08 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-08 Cosine Similarity between <추진> & <사

***2021-09-10 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-09-10 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2021-09-10 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-10 Cosine Similarity between <추진> & <발전> : 0.5099795460700989
***2021-09-10 Cosine Similarity between <해결> & <방안> : 0.5310750603675842
***2021-09-10 Cosine Similarity between <해결> & <문제> : 0.6186186075210571
***2021-09-10 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-09-10 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-09-10 Cosine Similarity between <본격적인> & <본격> : 0.6104379296302795
***2021-09-10 Cosine Similarity between <장점> & <단점> : 0.7721676826477051
***2021-09-10 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-10 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-09-10 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2021-09-10 Cosine Similarity between <추진> & <진

***2021-09-13 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-09-13 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-13 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-09-13 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-13 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-09-13 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-09-13 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-09-13 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-09-13 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-13 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2021-09-13 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-09-13 Cosine Similarity between <장점> & <특징> : 0.6442348957061768
***2021-09-13 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-13 Cosine Similarity between <증대> & <감소>

***2021-09-14 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-09-14 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-09-14 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-09-14 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-14 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-14 Cosine Similarity between <장점> & <특징> : 0.6442348957061768
***2021-09-14 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-09-14 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-14 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-14 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-09-14 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2021-09-14 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-14 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-09-14 Cosine Similarity between <추진> & <진행

***2021-09-15 Cosine Similarity between <추진> & <중인> : 0.539746880531311
***2021-09-15 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2021-09-15 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-15 Cosine Similarity between <해결> & <극복> : 0.5064623951911926
***2021-09-15 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2021-09-15 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-09-15 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-09-15 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-09-15 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-09-15 Cosine Similarity between <추진> & <계획> : 0.5205425024032593
***2021-09-15 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-09-15 Cosine Similarity between <추진> & <발전> : 0.5099795460700989
***2021-09-15 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2021-09-15 Cosine Similarity between <본격적인> & <본격>

In [11]:
pos_lexicon_2021

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2021-01-04,1,가치 있는,언택트,0.549658,1
1,2021-01-04,1,개선,향상,0.587848,1
2,2021-01-04,1,도움이 되는,언택트,0.537873,1
3,2021-01-04,1,우세,강세,0.512041,1
4,2021-01-04,1,의심의 여지가 없는,언택트,0.529306,1
...,...,...,...,...,...,...
28928,2021-09-16,5769,증대,증가,0.694643,1
28929,2021-09-16,5769,증대,확대,0.519324,2
28930,2021-09-16,5769,최선,최선,1.000000,1
28931,2021-09-16,5769,추진,계획,0.520543,1


In [12]:
pos_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/pos_samsung_2021.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2021['news_word'])))

a

['이유섭',
 '홈트레이딩',
 '침구',
 '아웃폴딩폰',
 '최찬기',
 '법정상속',
 '여의도고',
 '마사츠',
 '펫테크',
 '장점',
 '프터스',
 '듀센바예프',
 '프론트트론',
 '본격',
 '게이밍존',
 '트렌천트',
 '씨엠비나',
 '샹들리',
 '잔손질',
 '개선',
 '갤럴시',
 '박학규',
 '유펭두잇',
 '최고급',
 '특성',
 '입관식',
 '에임트',
 '리뉴어블',
 '동물의숲',
 '재택하',
 '간펑리튬',
 '이점',
 '빅프라핏',
 '뭉갠다',
 '강력',
 '스캠성',
 '코람코',
 '청맹과니',
 '머니무브',
 '곽현희',
 '리뉴텍',
 '겹악재',
 '쯔광궈신',
 '세리프체',
 '계산통계학',
 '의미',
 '맞소송',
 '겐로코아쿠',
 '자이앤트',
 '국립기관',
 '가팔랐으',
 '레그단비맙',
 '빅테크',
 '요구',
 '텔레칩스텔레',
 '블르투스',
 '액티브펀드',
 '온택트란',
 '이주철',
 '스타벤처스',
 '진행',
 '마련',
 '최길현',
 '사하나',
 '그린팹',
 '차익',
 '법무조합',
 '백혜진',
 '텍오스코텍',
 '안전컨설팅',
 '컴패티블',
 '해법',
 '언택트',
 '쎄미켐동진쎄미켐',
 '거버넌스위',
 '스트레티',
 '롱숏거래',
 '사업',
 '윈지코리아컨설팅',
 '구현',
 '가로본능폰',
 '갤탭용',
 '엡시헬스',
 '컨설턴츠',
 '문용주',
 '스플런크',
 '손희택씨',
 '팽팽하',
 '최진명',
 '플래트너',
 '경의',
 '저하',
 '레브친',
 '웰바이오텍',
 '발전',
 '호매실고',
 '취합하',
 '한승협',
 '박닌공장',
 '확대',
 '퀀텀오케스트',
 '극복',
 '코세스',
 '폐쇄됐으',
 '빠듯하',
 '휼렛패커드',
 '랙티브',
 '황예슬',
 '삐뚜름하',
 '훈풍',
 '언더포펌',
 '조윤오',
 '게임스탑',
 '안태혁',
 '디애틀레틱스',
 '자공시',
 '쟁점대검',
 

In [14]:
b = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['안전컨설팅',
 '충족',
 '실패',
 '약세',
 '추진',
 '경의',
 '최선',
 '밑돈',
 '단점',
 '장점',
 '쎄미켐동진쎄미켐',
 '우월',
 '밸프스챌린',
 '불황',
 '개선']

In [15]:
c = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['지성규',
 '최선',
 '텍옵트론텍',
 '감소',
 '충족',
 '빅프라핏',
 '콘엘비세미콘',
 '증가',
 '추진',
 '스타벤처스',
 '장점',
 '웃돌',
 '적극',
 '실패',
 '트렌천트',
 '경의',
 '밑돈',
 '단점',
 '세리프체',
 '최길학',
 '불황',
 '개선',
 '안전컨설팅',
 '맞소송',
 '약세',
 '송출되',
 '것이므',
 '쎄미켐동진쎄미켐',
 '우월',
 '밸프스챌린']